In [1]:
# Import Library
import numpy as np
import pandas as pd
#!pip install keras_tuner
#import keras_tuner

import keras
from keras import models, layers, metrics, Sequential
from keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf
from keras.regularizers import l1,l2,l1_l2
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate
from tensorflow.keras.models import Model

import tensorflow as tf
import random

#!pip install keras_tuner
import keras_tuner
from keras_tuner import HyperModel
import keras_tuner as kt

# Import Dataset

In [2]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
       'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
       'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
       'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
       'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']
info_list = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
       'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]
macro_col = ['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx','INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S','IPFUELS', 'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV',
       'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV','UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD',
       'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP','SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT',
       'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE','HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'AMDMNOx', 'ANDENOx', 'AMDMUOx', 'BUSINVx','ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500','S&P: indust', 'S&P div yield', 'S&P PE ratio', 'FEDFUNDS',
       'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA','COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM',
       'AAAFFM', 'BAAFFM', 'TWEXAFEGSMTHx', 'EXSZUSx', 'EXJPUSx','EXUSUKx', 'EXCAUSx', 'WPSFD49207', 'WPSFD49502', 'WPSID61',
       'WPSID62', 'OILPRICEx', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL','CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD',
       'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5','PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA',
       'CES0600000008', 'CES2000000008', 'CES3000000008', 'UMCSENTx','DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'VIXCLSx', 'dp', 'ep_macro', 'b/m',
       'ntis', 'tbl', 'tms', 'dfy', 'svar']


# Import Dataset
# path = '/content/drive/MyDrive/Colab Notebooks/Paper1/'
path= "dataset/"
firm_df = pd.read_pickle(path+'firm_df49.pkl')
firm_df.index = pd.to_datetime(firm_df.index).to_period('M')

# macro-economic variables
# There is one features which has 37 missing values but before year 1980
macro_df = pd.read_pickle(path+'macro_df.pkl')
macro_df.index = pd.to_datetime(macro_df.index, format='%m/%d/%Y')
macro_df.rename(columns={"ep": "ep_macro"}, inplace=True)
macro_index = macro_df.index[1:]
macro_new = macro_df.iloc[:-1, :]
macro_new.index = macro_index
macro_new.index = macro_new.index.to_period('M')

# Scale macro-features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
macro_scaled = scaler.fit_transform(macro_new)
macro_scaled_df = pd.DataFrame(macro_scaled, columns=macro_new.columns)
macro_scaled_df.index = macro_new.index

# merged data 1
merged_df = pd.merge(firm_df, macro_scaled_df, left_on='jdate', right_on='sasdate', how='inner')
merged_df.index = firm_df.index

In [3]:
class DataPrepare():
  def __init__(self):
    self.char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
       'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
       'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
       'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
       'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']
    self.info_list = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
       'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]
    self.macro_col = ['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx','INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S','IPFUELS', 'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV',
       'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV','UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD',
       'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP','SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT',
       'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE','HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'AMDMNOx', 'ANDENOx', 'AMDMUOx', 'BUSINVx','ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500','S&P: indust', 'S&P div yield', 'S&P PE ratio', 'FEDFUNDS',
       'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA','COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM',
       'AAAFFM', 'BAAFFM', 'TWEXAFEGSMTHx', 'EXSZUSx', 'EXJPUSx','EXUSUKx', 'EXCAUSx', 'WPSFD49207', 'WPSFD49502', 'WPSID61',
       'WPSID62', 'OILPRICEx', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL','CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD',
       'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5','PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA',
       'CES0600000008', 'CES2000000008', 'CES3000000008', 'UMCSENTx','DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'VIXCLSx', 'dp', 'ep_macro', 'b/m',
       'ntis', 'tbl', 'tms', 'dfy', 'svar']

  def datasplit(self, df, dataset):
    df_train = df[df.index < pd.Period((str(1995)+"-1"),freq='M')]
    df_val = df[(df.index >= pd.Period((str(1995)+"-1"),freq='M')) & (df.index < pd.Period((str(2000)+"-1"),freq='M'))]
    df_test = df[df.index >= pd.Period((str(2000)+"-1"),freq='M')]
    if dataset == "firm":
      X_train = df_train[self.char_core]
      X_val = df_val[self.char_core]
      X_test = df_test[self.char_core]
    elif dataset == "firm_macro":
      X_train = df_train[self.char_core+self.macro_col]
      X_val = df_val[self.char_core+self.macro_col]
      X_test = df_test[self.char_core+self.macro_col]
    y_train = df_train['predicted_return']
    y_val = df_val['predicted_return']
    y_test = df_test['predicted_return']
    return (X_train, y_train, X_val, y_val, X_test, y_test, )

In [4]:
prepare = DataPrepare()
X_train, y_train, X_val, y_val, X_test, y_test = prepare.datasplit(merged_df, dataset = "firm_macro")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 183) (88347,) (31851, 183) (31851,) (90007, 183) (90007,)


In [5]:
merged_train_firm = X_train[char_core]
merged_train_macro = X_train[macro_col]
merged_train_y = y_train
print(merged_train_firm.shape, merged_train_macro.shape, merged_train_y.shape)

merged_val_firm = X_val[char_core]
merged_val_macro = X_val[macro_col]
merged_val_y = y_val
print(merged_val_firm.shape, merged_val_macro.shape, merged_val_y.shape)

merged_test_firm = X_test[char_core]
merged_test_macro = X_test[macro_col]
merged_test_y = y_test
print(merged_test_firm.shape, merged_test_macro.shape, merged_test_y.shape)

(88347, 49) (88347, 134) (88347,)
(31851, 49) (31851, 134) (31851,)
(90007, 49) (90007, 134) (90007,)


In [6]:
def my_metric_fn(y_true, y_pred):
  num = tf.reduce_mean(tf.square(y_true - y_pred))
  den = tf.reduce_mean(tf.square(y_true - tf.zeros_like(y_true)))
  return 1 - num / den


def multi_input_model():
  # Define the first input with 134 features
  input1 = Input(shape=(134,))
  x1 = Dense(4, activation='relu')(input1)
  x1 = Dense(49, activation='relu')(x1)

  input2 = Input(shape=(49,))

  # Multiply the outputs of the first branch and the second input element-wise
  merged = Multiply()([x1, input2])

  # Create the second branch of the network
  x2 = Dense(32, activation='relu')(merged)
  x2 = Dense(16, activation='relu')(x2)
  output = Dense(1, activation='linear')(x2)

  # Create the model with two inputs and one output
  model = Model(inputs=[input1, input2], outputs=output)
  # optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)
  optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001, beta_1=0.92) # best
  model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[my_metric_fn])
  return model

In [7]:
print(merged_train_firm.shape, merged_train_macro.shape, merged_train_y.shape)
# Create the model
random_seed = 1120
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
model = multi_input_model()

# Fit the model to your training data
model.fit([merged_train_macro, merged_train_firm], merged_train_y,
          validation_data=([merged_val_macro, merged_val_firm], merged_val_y),
          epochs=100,
          batch_size=128,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=3)],
          )  # You can adjust the number

(88347, 49) (88347, 134) (88347,)
Epoch 1/100
691/691 [==============================] - 1s 819us/step - loss: 0.0121 - my_metric_fn: 0.0043 - val_loss: 0.0146 - val_my_metric_fn: 0.0043
Epoch 2/100
691/691 [==============================] - 0s 668us/step - loss: 0.0120 - my_metric_fn: 0.0112 - val_loss: 0.0146 - val_my_metric_fn: 0.0037
Epoch 3/100
691/691 [==============================] - 0s 585us/step - loss: 0.0119 - my_metric_fn: 0.0219 - val_loss: 0.0147 - val_my_metric_fn: -0.0014
Epoch 4/100
691/691 [==============================] - 1s 1ms/step - loss: 0.0118 - my_metric_fn: 0.0347 - val_loss: 0.0148 - val_my_metric_fn: -0.0069


In [8]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()
# 10 Trials for Prediction
merged_train_macro_all = np.concatenate((merged_train_macro, merged_val_macro))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    model = multi_input_model()
    model.fit(x=[merged_train_macro_all, merged_train_firm_all], y=merged_y_all,
              epochs=30,
              batch_size = 128,
              validation_data=([merged_test_macro, merged_test_firm], merged_test_y),
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(model.evaluate([merged_test_macro, merged_test_firm], merged_test_y))
    y_hat = model.predict([merged_test_macro, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

Epoch 1/30
940/940 [==============================] - 1s 967us/step - loss: 0.0128 - my_metric_fn: 6.0141e-04 - val_loss: 0.0157 - val_my_metric_fn: 0.0059
Epoch 2/30
940/940 [==============================] - 1s 751us/step - loss: 0.0128 - my_metric_fn: 0.0039 - val_loss: 0.0156 - val_my_metric_fn: 0.0064
Epoch 3/30
2813/2813 [==============================] - 1s 386us/step - loss: 0.0157 - my_metric_fn: -0.0012
[0.0156911201775074, -0.0012159886537119746]
2813/2813 [==============================] - 1s 299us/step

Epoch 1/30
940/940 [==============================] - 1s 1ms/step - loss: 0.0128 - my_metric_fn: 0.0042 - val_loss: 0.0156 - val_my_metric_fn: 0.0065
Epoch 2/30
2813/2813 [==============================] - 1s 248us/step - loss: 0.0156 - my_metric_fn: 7.0308e-04
[0.01560688205063343, 0.0007030786946415901]
2813/2813 [==============================] - 1s 260us/step

Epoch 1/30
940/940 [==============================] - 2s 1ms/step - loss: 0.0128 - my_metric_fn: 0.0031 - val_l

In [9]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.009153514814708386


In [10]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()
# 10 Trials for Prediction
merged_train_macro_all = np.concatenate((merged_train_macro, merged_val_macro))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    model = multi_input_model()
    model.fit([merged_train_macro, merged_train_firm], merged_train_y,
              validation_data=([merged_val_macro, merged_val_firm], merged_val_y),
              epochs=30,
              batch_size = 128,
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)]
              )
    print(model.evaluate([merged_test_macro, merged_test_firm], merged_test_y))
    y_hat = model.predict([merged_test_macro, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

Epoch 1/30
691/691 [==============================] - 2s 2ms/step - loss: 0.0121 - my_metric_fn: -8.0408e-05 - val_loss: 0.0146 - val_my_metric_fn: 0.0035
Epoch 2/30
691/691 [==============================] - 0s 687us/step - loss: 0.0121 - my_metric_fn: 0.0048 - val_loss: 0.0146 - val_my_metric_fn: 0.0043
Epoch 3/30
2813/2813 [==============================] - 1s 264us/step - loss: 0.0157 - my_metric_fn: -6.2087e-04
[0.015727000311017036, -0.0006208717823028564]
2813/2813 [==============================] - 1s 235us/step

Epoch 1/30
691/691 [==============================] - 2s 3ms/step - loss: 0.0121 - my_metric_fn: 0.0033 - val_loss: 0.0146 - val_my_metric_fn: 0.0048
Epoch 2/30
2813/2813 [==============================] - 2s 575us/step - loss: 0.0156 - my_metric_fn: 0.0040
[0.015611090697348118, 0.003969299606978893]
2813/2813 [==============================] - 1s 324us/step

Epoch 1/30
691/691 [==============================] - 1s 1ms/step - loss: 0.0121 - my_metric_fn: 0.0024 - val_

In [11]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.008230163432272009


In [12]:
## Having patience = 2 is worse, which gives R^2 == 0.006273572885498457

seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()
# 10 Trials for Prediction
merged_train_macro_all = np.concatenate((merged_train_macro, merged_val_macro))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    model = multi_input_model()
    model.fit([merged_train_macro, merged_train_firm], merged_train_y,
              validation_data=([merged_val_macro, merged_val_firm], merged_val_y),
              epochs=30,
              batch_size = 128,
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=2)]
              )
    print(model.evaluate([merged_test_macro, merged_test_firm], merged_test_y))
    y_hat = model.predict([merged_test_macro, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

Epoch 1/30
691/691 [==============================] - 2s 2ms/step - loss: 0.0121 - my_metric_fn: -8.0408e-05 - val_loss: 0.0146 - val_my_metric_fn: 0.0035
Epoch 2/30
691/691 [==============================] - 0s 591us/step - loss: 0.0121 - my_metric_fn: 0.0048 - val_loss: 0.0146 - val_my_metric_fn: 0.0043
Epoch 3/30
691/691 [==============================] - 0s 569us/step - loss: 0.0120 - my_metric_fn: 0.0130 - val_loss: 0.0147 - val_my_metric_fn: -0.0013
Epoch 4/30
2813/2813 [==============================] - 1s 323us/step - loss: 0.0158 - my_metric_fn: -0.0148
[0.015846000984311104, -0.01483175065368414]
2813/2813 [==============================] - 1s 241us/step

Epoch 1/30
691/691 [==============================] - 1s 710us/step - loss: 0.0121 - my_metric_fn: 0.0033 - val_loss: 0.0146 - val_my_metric_fn: 0.0048
Epoch 2/30
691/691 [==============================] - 1s 1ms/step - loss: 0.0120 - my_metric_fn: 0.0104 - val_loss: 0.0146 - val_my_metric_fn: 0.0023
Epoch 3/30
2813/2813 [==

# Finished ---

In [77]:
# lr = 0.0005 beta_1 = 0.95
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

-0.04536933219533523


In [72]:
# beta_1 = 0.95
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.009117545377412162


In [67]:
# beta_1 = 0.92
#  Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = npx.mean(np.square(y_real))
print(1-a/b)

0.009153514814708386


In [63]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.009135297134495346


In [57]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.007872937845400108


In [30]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.009098028226459998


In [27]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.007956089805917554


# Try 2

In [ ]:
def my_metric_fn(y_true, y_pred):
  num = tf.reduce_mean(tf.square(y_true - y_pred))
  den = tf.reduce_mean(tf.square(y_true - tf.zeros_like(y_true)))
  return 1 - num / den


def multi_input_model2():
  # Define the first input with 134 features
  input1 = Input(shape=(134,))

  # Define the first branch of the network
  x1 = Dense(128, activation='relu')(input1)
  x1 = Dense(64, activation='relu')(x1)
  x1 = Dense(49, activation='relu')(x1)

  # Define the second input with 49 features
  input2 = Input(shape=(49,))

  # Multiply the outputs of the first branch and the second input element-wise
  merged = Multiply()([x1, input2])

  # Create the second branch of the network
  output = Dense(1, activation='linear')(merged)

  # Create the model with two inputs and one output
  model = Model(inputs=[input1, input2], outputs=output)

  # Compile the model (you can adjust the optimizer and loss function as needed)
  model.compile(optimizer='adam', loss='mean_squared_error', metrics=[my_metric_fn])

  return model

model2 = multi_input_model2()
model2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 134)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 128)                  17280     ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 64)                   8256      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 49)                   3185      ['dense_1[0][0]']             
                                                                                              

In [ ]:
print(merged_train_firm.shape, merged_train_macro.shape, merged_train_y.shape)
# Create the model
random_seed = 1120
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
model2 = multi_input_model2()

# Fit the model to your training data
model2.fit([merged_train_macro, merged_train_firm], merged_train_y,
          validation_data=([merged_val_macro, merged_val_firm], merged_val_y),
          epochs=100,
          batch_size=128,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=3)],
          )  # You can adjust the number

(88347, 49) (88347, 134) (88347,)
Epoch 1/100
691/691 [==============================] - 5s 5ms/step - loss: 0.0433 - my_metric_fn: -2.4062 - val_loss: 0.0151 - val_my_metric_fn: -0.0424
Epoch 2/100
691/691 [==============================] - 4s 6ms/step - loss: 0.0121 - my_metric_fn: 0.0018 - val_loss: 0.0146 - val_my_metric_fn: 0.0012
Epoch 3/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0121 - my_metric_fn: 0.0026 - val_loss: 0.0147 - val_my_metric_fn: -0.0023
Epoch 4/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0121 - my_metric_fn: 0.0027 - val_loss: 0.0146 - val_my_metric_fn: 0.0022
Epoch 5/100
691/691 [==============================] - 3s 4ms/step - loss: 0.0121 - my_metric_fn: 0.0027 - val_loss: 0.0146 - val_my_metric_fn: 5.7153e-04
Epoch 6/100
691/691 [==============================] - 5s 7ms/step - loss: 0.0121 - my_metric_fn: 0.0026 - val_loss: 0.0146 - val_my_metric_fn: 0.0021
Epoch 7/100
691/691 [==============================] 

In [ ]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()
# 10 Trials for Prediction
merged_train_macro_all = np.concatenate((merged_train_macro, merged_val_macro))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    model2.fit(x=[merged_train_macro_all, merged_train_firm_all], y=merged_y_all, epochs=20,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(model2.evaluate([merged_test_macro, merged_test_firm], merged_test_y))
    y_hat = model2.predict([merged_test_macro, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

Epoch 1/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 8.2239e-04
Epoch 2/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 7.5088e-04
Epoch 3/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 7.4576e-04
Epoch 4/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 7.9300e-04
Epoch 5/20
3757/3757 [==============================] - 10s 3ms/step - loss: 0.0128 - my_metric_fn: 9.2977e-04
Epoch 6/20
3757/3757 [==============================] - 11s 3ms/step - loss: 0.0128 - my_metric_fn: 0.0010
Epoch 7/20
2813/2813 [==============================] - 6s 2ms/step - loss: 0.0156 - my_metric_fn: 0.0069
[0.015640422701835632, 0.0068958960473537445]
2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
3757/3757 [==============================] - 10s 3ms/step - loss: 0.0128 - my_metric_fn: 7.0753e-04
Epoch 2/20
3757/3

In [ ]:
yhat_df

,458,165,530,564,590,560,829,170,376,176
0,0.008891,0.001587,0.010612,-0.001676,0.002848,0.004204,0.003156,0.00899,0.003406,0.002995
1,0.008891,0.001587,0.010612,-0.001676,0.002848,0.004204,0.003156,0.00899,0.003406,0.002995
2,0.008891,0.001587,0.010612,-0.001676,0.002848,0.004204,0.003156,0.00899,0.003406,0.002995
3,0.008891,0.001587,0.010612,-0.001676,0.002848,0.004204,0.003156,0.00899,0.003406,0.002995
4,0.008891,0.001587,0.010612,-0.001676,0.002848,0.004204,0.003156,0.00899,0.003406,0.002995
...,...,...,...,...,...,...,...,...,...,...
90002,0.008891,0.001587,0.010612,-0.001676,0.002848,0.004204,0.003156,0.00899,0.003406,0.002995
90003,0.008891,0.001587,0.010612,-0.001676,0.002848,0.004204,0.003156,0.00899,0.003406,0.002995
90004,0.008891,0.001587,0.010612,-0.001676,0.002848,0.004204,0.003156,0.00899,0.003406,0.002995
90005,0.008891,0.001587,0.010612,-0.001676,0.002848,0.004204,0.003156,0.00899,0.003406,0.002995


In [ ]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.004926950787370377


In [ ]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.006030776913818592


In [ ]:
merged_train = merged_df[:120198]
merged_test = merged_df[120198:]

merged_train_firm = merged_train[char_core]
merged_train_macro = merged_train[macro_col]
merged_train_y = merged_train['predicted_return']

merged_test_firm = merged_test[char_core]
merged_test_macro = merged_test[macro_col]
merged_test_y = merged_test['predicted_return']

print(merged_train_firm.shape, merged_train_macro.shape, merged_train_y.shape)
print(merged_test_firm.shape, merged_test_macro.shape, merged_test_y.shape)

(120198, 49) (120198, 134) (120198,)
(90007, 49) (90007, 134) (90007,)


In [ ]:
def multi_input_model():
  # Define the first input with 134 features
  input1 = Input(shape=(134,))

  # Define the first branch of the network
  x1 = Dense(128, activation='relu')(input1)
  x1 = Dense(64, activation='relu')(x1)
  x1 = Dense(49, activation='relu')(x1)

  # Define the second input with 49 features
  input2 = Input(shape=(49,))

  # Multiply the outputs of the first branch and the second input element-wise
  merged = Multiply()([x1, input2])

  # Create the second branch of the network
  x2 = Dense(32, activation='relu')(merged)
  x2 = Dense(16, activation='relu')(x2)
  output = Dense(1, activation='linear')(x2)

  # Create the model with two inputs and one output
  model = Model(inputs=[input1, input2], outputs=output)

  # Compile the model (you can adjust the optimizer and loss function as needed)
  model.compile(optimizer='adam', loss='mean_squared_error')

  return model

model = multi_input_model()
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 134)]                0         []                            
                                                                                                  
 dense_30 (Dense)            (None, 128)                  17280     ['input_11[0][0]']            
                                                                                                  
 dense_31 (Dense)            (None, 64)                   8256      ['dense_30[0][0]']            
                                                                                                  
 dense_32 (Dense)            (None, 49)                   3185      ['dense_31[0][0]']            
                                                                                            

In [ ]:
print(merged_train_firm.shape, merged_train_macro.shape, merged_train_y.shape)
random_seed = 1120
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

# Create the model
model = multi_input_model()

# Fit the model to your training data
model.fit([merged_train_macro, merged_train_firm], merged_train_y, epochs=10, batch_size=128)  # You can adjust the number of epochs and batch size
# Evaluate the model on your test data
loss = model.evaluate([merged_test_macro, merged_test_firm], merged_test_y,)
print("Test Loss:", loss)
yhat = model.predict([merged_test_macro, merged_test_firm])

# Print out Predictive R^2
y_predict = yhat.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

(120198, 49) (120198, 134) (120198,)
Epoch 1/10
940/940 [==============================] - 6s 5ms/step - loss: 0.0178
Epoch 2/10
940/940 [==============================] - 4s 5ms/step - loss: 0.0127
Epoch 3/10
940/940 [==============================] - 3s 4ms/step - loss: 0.0125
Epoch 4/10
940/940 [==============================] - 3s 4ms/step - loss: 0.0124
Epoch 5/10
940/940 [==============================] - 5s 5ms/step - loss: 0.0122
Epoch 6/10
940/940 [==============================] - 4s 4ms/step - loss: 0.0122
Epoch 7/10
940/940 [==============================] - 4s 4ms/step - loss: 0.0121
Epoch 8/10
940/940 [==============================] - 3s 4ms/step - loss: 0.0121
Epoch 9/10
940/940 [==============================] - 5s 5ms/step - loss: 0.0121
Epoch 10/10
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0165
Test Loss: 0.016530562192201614
2813/2813 [==============================] - 7s 2ms/step
-0.04920983307311544


In [ ]:
# Evaluate the model on your test data
loss = model.evaluate([merged_test_macro, merged_test_firm], merged_test_y,)
print("Test Loss:", loss)
yhat = model.predict([merged_test_macro, merged_test_firm])

# Print out Predictive R^2
y_predict = yhat.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

2813/2813 [==============================] - 8s 3ms/step - loss: 0.0159
Test Loss: 0.015909628942608833
2813/2813 [==============================] - 19s 7ms/step
-0.009797561384419007


In [ ]:
# HyperParameter Tuning
class MyHyperModel(HyperModel):
  def __init__(self,batch_size, random_seed=None,):
    self.batch_size = batch_size
    if random_seed is not None:
      random.seed(random_seed)
      np.random.seed(random_seed)
      tf.random.set_seed(random_seed)

  def my_metric_fn(self, y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true - tf.zeros_like(y_true)))
    return 1 - num / den

  def call_existing_code(self,):
    # Define the first input with 134 features
    input1 = Input(shape=(134,))
    # Define the first branch of the network
    x1 = Dense(128, activation='relu')(input1)
    x1 = Dense(64, activation='relu')(x1)
    x1 = Dense(49, activation='relu')(x1)
    # Define the second input with 49 features
    input2 = Input(shape=(49,))
    # Multiply the outputs of the first branch and the second input element-wise
    merged = Multiply()([x1, input2])
    # Create the second branch of the network
    x2 = Dense(32, activation='relu')(merged)
    x2 = Dense(16, activation='relu')(x2)
    output = Dense(1, activation='linear')(x2)
    # Create the model with two inputs and one output
    model = Model(inputs=[input1, input2], outputs=output)
    # Compile the model (you can adjust the optimizer and loss function as needed)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

  def build(self, hp, *args, **kwargs):
    #l1_reg = hp.Choice("l1_ratio", self.l1_reg)
    #l2_reg = hp.Choice("l2_ratio", self.l2_reg)
    #layer = hp.Choice("layer", self.layer)
    #dropout = hp.Choice("dropout", self.dropout)
    #lr = hp.Choice("learning_rate", self.lr)
    model = self.call_existing_code()
    return model

  def fit(self, hp, model, *args, **kwargs):
    return model.fit(
    *args,
    batch_size=hp.Choice("batch_size", self.batch_size),
    epochs=self.epoch,
    **kwargs,
    )

In [ ]:
random_seed = 1120
tuner = kt.RandomSearch(
    MyHyperModel(random_seed=random_seed, batch_size = [128,],),
    objective="val_loss",
    max_trials=100,
    directory="my_dir",
    overwrite=True,
    project_name="tune_hypermodel",
)
tuner.search([merged_train_macro, merged_train_firm], merged_train_y,
             validation_data=([merged_val_macro, merged_val_firm], merged_val_y),
       callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

# Print best parameters
best_hps = tuner.get_best_hyperparameters(5)
hps1_dic = best_hps[0].values
print(hps1_dic)

Trial 1 Complete [00h 00m 01s]

Best val_loss So Far: None
Total elapsed time: 00h 00m 01s
{'batch_size': 128}
